## Project 1: Class Scheduling with Mathematical Programming

* **Collaborators**: Isidore Sossa and Chase Scott
* **Resources**: 
    * Nurses Scheduling Problem
* **Project's requirements**:
    * Use data provided through Canvas
    * Generate random list of preferences for each professor according to their qualification, i.e. an integer from 1 to *n*, where *n* is the number of classes the professor is qualified to teach. The higher the number the lesser the preference.
    * Extract out professors who do not teach
    * Professors who do teach work at the maximum of their workload
    * Classes start at 8:00 AM and end at 6:00 PM and can only start at every half period.
    * One credit hour is equivalent to 50 minutes. E.g. A 3-credit hour class meet for a total of 150 minutes per week.
    * Add two constraints of your chosen

In [1]:
import pandas as pd
from docplex.mp.model import Model
import math
import random

### Functions and Customizations

In [2]:
## Standardize data frame column names
def clean_dataframe_columns(df:pd.DataFrame) -> pd.DataFrame:
    colnames = df.columns.values.tolist()
    colnames = [col.replace(' ', '_') for col in colnames]
    df.columns = colnames

In [3]:
## Duplicate courses with multiple sessions
def standardize_courses_offered(courses_offered:pd.DataFrame, course_catalog_df:pd.DataFrame) -> pd.DataFrame:
    ''''''
    def get_day_name(day:str):
        days_name = {'M':'Monday', 'TU':'Tuesday', 'W':'Wednesday', 'TH':'Thursday', 'F':'Friday'}
        return days_name[day]
    
    result = pd.DataFrame()
    result = result.append(courses_offered[courses_offered['Sections'] == 1])
    for course in courses_offered[courses_offered['Sections'] > 1].itertuples():
        course = pd.DataFrame(course).T.drop(columns=0)
        course.columns = courses_offered.columns
        for i in range(1, course['Sections'][0] + 1):
            course['Sections'] = i
            result = result.append([course], ignore_index=True)
    result = result.rename(columns={'Sections' : 'Session'})
    result = result.sort_values(by=['Course_Number', 'Session']).reset_index().drop(columns='index')
    result = pd.merge(left=result, right=course_catalog_df, on=['Department_Code', 'Course_Number', 'Course_Name'])
    
    courses_offered = result
    result = pd.DataFrame()
    
    for course in courses_offered.itertuples():
        course = pd.DataFrame(course).T.drop(columns=0)
        course.columns = courses_offered.columns
        days_taught = course['Day'][0]
        days_taught = days_taught.split('/')
        for i in days_taught:
            course['Day'] = i
            result = result.append([course], ignore_index=True)
            
    result['Day'] = result.apply(lambda row : get_day_name(row.Day), axis=1)
    
    periods_covered = []
    courses = []
    sessions = []
    days = []
    CLASS_DURATION = 50
    PERIOD_DURATION = 30
    for course in result.itertuples():
        course_name, session = course.Course_Name, course.Session
        number_of_credit = course.Credit_Hours
        day = course.Day
        number_of_weekly_meetings = len(result[(result.Course_Name == course_name) & (result.Session == session)])
        duration = (number_of_credit * CLASS_DURATION) / (number_of_weekly_meetings * PERIOD_DURATION)
        number_of_period_covered = math.ceil(duration)
        periods_covered.append(number_of_period_covered)
        courses.append(course_name)
        sessions.append(session)
        days.append(day)

    temp_df = pd.DataFrame({'Course_Name' : courses, 'Session' : sessions, 'Number_Periods_Covered' : periods_covered, 
                           'Day' : days})
    result = pd.merge(left=result, right=temp_df, on=['Course_Name', 'Session', 'Day'])
    
    return result

In [4]:
def merge_and_get_preference(professors_df:pd.DataFrame, qualifications:pd.DataFrame, course:str) -> pd.DataFrame:
    ''''''
    def manipulate_row(row:pd.DataFrame, course:str):
        cols = [x for x in row.columns if course in x]
        class_taught = 0
        for col in cols:
            if not math.isnan(row[col][0]):
                class_taught += row[col][0]
        class_taught = int(class_taught)
        if row['Workload_Credit_Hours'][0] > 0:
            preferences = random.sample(list(range(1, class_taught + 1)), class_taught)
        else:
            preferences = [0] * class_taught
            

        index = 0
        for col in cols:
            if not math.isnan(row[col][0]):
                row[col] = preferences[index]
                index += 1
            else:
                row[col] = 0
        return row

    result_df = pd.merge(left=professors_df, right=qualifications_df, on=['Faculty_Name', 'Department_Code'])
    col_names = list(result_df.columns)
    preferences_df = pd.DataFrame(columns=col_names)
    col_names = result_df.columns
    for row in result_df.itertuples():
        row_df = pd.Series(row).to_frame().T.drop(columns=0)
        row_df.columns = col_names
        row_df = manipulate_row(row_df, course)
        preferences_df = preferences_df.append(row_df, ignore_index=True)
    return preferences_df

In [5]:
def get_period_df(start_hour=8, end_hour=18) -> pd.DataFrame:
    '''Create a dataframe with days'''
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    days_of_weeks = dict(zip(days, range(0, 5)))
    days_of_weeks

    result = pd.DataFrame(columns=['Period', 'Day'])
    start = 1
    day_digits = []
    day_names = []
    for day, day_digit in days_of_weeks.items():
        day_names.extend([day] * 20)
        day_digits.extend([day_digit] * 20)
        
    start_time = [x * 0.5 for x in range(2*start_hour, 2*end_hour)] * len(days)
    end_time = [x * 0.5 for x in range(2*start_hour + 1, 2*end_hour + 1)] * len(days)
    
    
    return pd.DataFrame.from_dict({'Period' : list(range(1, 101)), 'Day_Name' : day_names, 'Day_Number' : day_digits, 
                                   'Start_Time' : start_time, 'End_Time' : end_time})


In [6]:
def get_column_value_as_list(df:pd.DataFrame, colname:str) -> pd.DataFrame:
    '''Get the values of a data frame column as a Python list'''
    if colname in df.columns:
        return df[colname].values.tolist()
    else:
        return None

### Data Exploration and Cleaning

In [7]:
## Load data
data_file = 'Data Set – Class Schedule.xlsx'
file_handler = pd.ExcelFile(path_or_buffer=data_file)
file_sheets = sorted(file_handler.sheet_names)
print(f'Sheets : {file_sheets}')

## Load each sheet as a DataFrame
classrooms_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[0])
clean_dataframe_columns(classrooms_df)
display(file_sheets[0], classrooms_df)

colleges_and_depts = pd.read_excel(io=file_handler, sheet_name=file_sheets[1])
clean_dataframe_columns(colleges_and_depts)
display(file_sheets[1], colleges_and_depts.head(5))

course_catalog_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[2])
clean_dataframe_columns(course_catalog_df)
display(file_sheets[2], course_catalog_df.sort_values('Course_Name').head(25))

courses_offered = pd.read_excel(io=file_handler, sheet_name=file_sheets[3])
clean_dataframe_columns(courses_offered)
display(file_sheets[3], courses_offered.sort_values('Course_Name').head(45))

professors_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[4])
clean_dataframe_columns(professors_df)
display(file_sheets[4], professors_df.head(5))

qualifications_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[5])
clean_dataframe_columns(qualifications_df)
display(file_sheets[5], qualifications_df.head(5))

Sheets : ['Classrooms', 'Colleges and Departments', 'Course Catalog', 'Courses Offered Fall 2021', 'Professors', 'Qualification']


'Classrooms'

,Building,Building_Code,Classroom,Capacity
0,Durland,DU,1029,47
1,Durland,DU,1041,24
2,Durland,DU,1052,60
3,Durland,DU,1061,47
4,Durland,DU,1063,47
5,Durland,DU,1066,91
6,Durland,DU,1069,22
7,Durland,DU,1073,185


'Colleges and Departments'

,College,College_Code,Department,Department_Code
0,College of Agriculture,CAGRI,Entomology,ENTOM


...

,Department_Code,Course_Number,Course_Name,Sections,Day,Max_Enrollement
25,ENTOM,835,Advanced Insect Evolution,1,M/W/F,20
23,ENTOM,825,Advanced Integrative Behavioral Ecology,1,M/W,20
2,ENTOM,305,Animal Health Entomology,3,TU/TH,50
3,ENTOM,306,Animal Health Entomology Laboratory,1,M/W/F,50
15,ENTOM,680,Aquatic Entomology,1,M/W/F,40
28,ENTOM,849,Arthropod Vectors of Human Pathogens,1,M/W/F,20
33,ENTOM,885,Conventional and Molecular Methods for Evaluat...,1,M/W,20
4,ENTOM,350,"Crops, Insects, and Agroecosystems",1,M/W,50
32,ENTOM,880,Ecological Genomics,1,M/TU/W/TH,20
0,ENTOM,300,Economic Entomology,1,M/W/F,50


'Professors'

,Department_Code,Faculty_Name,Workload_Credit_Hours
0,ENTOM,Frank H. Arthur,6
1,ENTOM,James F. Campbell,9
2,ENTOM,Ming-Shun Chen,9
3,ENTOM,Raymond Cloyd,9
4,ENTOM,Lee Cohnstaedt,9


'Qualification'

,Department_Code,Faculty_Name,ENTOM_300,ENTOM_301,ENTOM_305,ENTOM_306,ENTOM_350,ENTOM_589,ENTOM_602,ENTOM_621,...,ENTOM_830,ENTOM_835,ENTOM_837,ENTOM_840,ENTOM_849,ENTOM_857,ENTOM_860,ENTOM_875,ENTOM_880,ENTOM_885
0,ENTOM,Frank H. Arthur,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,...,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,ENTOM,James F. Campbell,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0
2,ENTOM,Ming-Shun Chen,NaN,1.0,1.0,1.0,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
3,ENTOM,Raymond Cloyd,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,...,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
4,ENTOM,Lee Cohnstaedt,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,...,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN


In [8]:
# For reproducibility
random.seed(3)

courses_offered = standardize_courses_offered(courses_offered, course_catalog_df)
preferences_df = merge_and_get_preference(professors_df, qualifications_df, 'ENTOM')
display(preferences_df)
periods_df = get_period_df()
display(periods_df.head(21))

,Department_Code,Faculty_Name,Workload_Credit_Hours,ENTOM_300,ENTOM_301,ENTOM_305,ENTOM_306,ENTOM_350,ENTOM_589,ENTOM_602,...,ENTOM_830,ENTOM_835,ENTOM_837,ENTOM_840,ENTOM_849,ENTOM_857,ENTOM_860,ENTOM_875,ENTOM_880,ENTOM_885
0,ENTOM,Frank H. Arthur,6,0,4,3,0,8,0,0,...,7,0,0,6,2,0,0,0,0,0
1,ENTOM,James F. Campbell,9,0,5,8,0,2,6,0,...,0,0,0,0,3,0,0,0,7,1
2,ENTOM,Ming-Shun Chen,9,0,3,4,6,0,2,0,...,0,0,0,0,0,0,0,9,1,7
3,ENTOM,Raymond Cloyd,9,0,3,10,1,5,11,0,...,0,6,0,0,2,8,0,0,0,0
4,ENTOM,Lee Cohnstaedt,9,10,8,3,6,1,0,7,...,11,0,0,0,5,0,9,0,0,0
5,ENTOM,Srinivas Kambhampati,6,0,0,5,7,0,0,0,...,0,0,0,6,0,0,2,0,1,0
6,ENTOM,Tania Kim,3,3,0,6,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,ENTOM,Berlin Luxelly Londono Renteria,6,5,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
8,ENTOM,Jeremy L. Marshall,9,5,1,7,4,0,0,8,...,0,0,0,0,0,0,0,0,0,0
9,ENTOM,Brian P. McCornack,9,0,0,0,7,0,0,0,...,4,8,0,6,5,0,0,0,0,0


,Period,Day_Name,Day_Number,Start_Time,End_Time
0,1,Monday,0,8.0,8.5
1,2,Monday,0,8.5,9.0
2,3,Monday,0,9.0,9.5
3,4,Monday,0,9.5,10.0
4,5,Monday,0,10.0,10.5
5,6,Monday,0,10.5,11.0
6,7,Monday,0,11.0,11.5
7,8,Monday,0,11.5,12.0
8,9,Monday,0,12.0,12.5
9,10,Monday,0,12.5,13.0


### Mathematical Model

#### Model Initialization

In [9]:
## Model Environment
model = Model(name='Course Scheduling')

######################
# Decision Variables #
######################
x_fcsrp = dict()
for professor in preferences_df.Faculty_Name:
    for course in courses_offered.itertuples():
        for period in periods_df[periods_df.Day_Name == course.Day].Period.values.tolist():
            for room in classrooms_df.Classroom:
                x_fcsrp[(professor, course.Course_Name, course.Session, room, period)] = model.binary_var(name = "x({},{},{},{},{})".format(professor, course.Course_Name, course.Session, room, period))

model.print_information()

Model: Course Scheduling
 - number of variables: 397440
   - binary=397440, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


#### Constraints

In [10]:
### Constants (in minutes)
CLASS_DURATION = 50
PERIOD_DURATION = 30

### Exclude faculty member whose workload is zero
for professor in preferences_df[preferences_df.Workload_Credit_Hours == 0].itertuples():
    for course in courses_offered.itertuples():
        for period in periods_df[periods_df.Day_Name == course.Day].Period.values.tolist():
            for room in classrooms_df.Classroom:
                model.add_constraint(x_fcsrp[(professor.Faculty_Name, course.Course_Name, course.Session, room, period)] == 0)

### Within a given time frame, a faculty can teach at most one course in one room
for professor in preferences_df.Faculty_Name:
    for days in courses_offered.groupby(by='Day'):
        periods = periods_df[periods_df.Day_Name == days[0]].Period.values.tolist()
        for period_covered in days[1].Number_Periods_Covered.values.tolist():
            intervals = zip(periods, periods[period_covered:])
            for interval in intervals:
                start = interval[0]
                end = interval[1]
                aggregate_values = []
                for period in range(start, end):
                    for course in days[1].itertuples():
                        course_name, session = course.Course_Name, course.Session
                        for room in classrooms_df.Classroom:
                            aggregate_values.append(x_fcsrp[(professor, course.Course_Name, course.Session, room, period)])
                model.add_constraint(model.sum(aggregate_values) <= 1)

### Only one room can host a class in any given time
for room in classrooms_df.Classroom:
    for days in courses_offered.groupby(by='Day'):
        periods = periods_df[periods_df.Day_Name == days[0]].Period.values.tolist()
        for period_covered in days[1].Number_Periods_Covered.values.tolist():
            intervals = zip(periods, periods[period_covered:])
            for interval in intervals:
                start = interval[0]
                end = interval[1]
                aggregate_values = []
                for period in range(start, end):
                    for course in days[1].itertuples():
                        course_name, session = course.Course_Name, course.Session
                        for professor in preferences_df.Faculty_Name:
                            aggregate_values.append(x_fcsrp[(professor, course_name, session, room, period)])
                model.add_constraint(model.sum(aggregate_values) <= 1)

## Each course is assigned to one professor in one room
for course in courses_offered.groupby(by=['Course_Name', 'Session']):
    course_name, session = course[0]
    number_of_period_covered = course[1].Number_Periods_Covered.values.tolist()[0]
    for day_offered in course[1].Day.values.tolist():
        aggregate_values = []
        periods = periods_df[periods_df.Day_Name == day_offered].Period.values.tolist()
        for index, period in enumerate(periods):
            for professor in preferences_df.Faculty_Name:
                for room in classrooms_df.Classroom:
                    if index > len(periods) - number_of_period_covered:
                        model.add_constraint(x_fcsrp[(professor, course_name,session, room, period)] == 0)
                    aggregate_values.append(x_fcsrp[(professor, course_name,session, room, period)])
        model.add_constraint(model.sum(aggregate_values) == 1)
        
### Each course is taught at the same time in the same room by the same faculty on the day it is offered
for course in courses_offered.groupby(by=['Course_Name', 'Session']):
    course_name, session = course[0]
    days_offered = course[1].Day.values.tolist()
    for index in range(len(days_offered) - 1):
        x = periods_df[periods_df.Day_Name == days_offered[index]].Period.values.tolist()
        y = periods_df[periods_df.Day_Name == days_offered[index + 1]].Period.values.tolist()
        for periods in list(zip(x, y)):
            for professor in preferences_df.Faculty_Name:
                for room in classrooms_df['Classroom']:
                    aggregate_values = []
                    aggregate_values.append(x_fcsrp[(professor, course_name, session, room, periods[0])])
                    aggregate_values.append(x_fcsrp[(professor, course_name, session, room, periods[1])])
                    model.add_constraint(aggregate_values[0] - aggregate_values[1] == 0)

### A course is taught in a classroom that can hold at least the maximum number of students enrolled
for course in courses_offered.itertuples():
    for period in periods_df[periods_df.Day_Name == course.Day].Period.values.tolist():
        for professor in preferences_df.Faculty_Name:
            for room in classrooms_df.itertuples():
                model.add_constraint(x_fcsrp[(professor, course.Course_Name, course.Session, room.Classroom, period)] * course.Max_Enrollement <= room.Capacity)

### Each faculty member can only teach up to his or her workload
for professor in preferences_df.itertuples():
    aggregate_values = []
    for course in courses_offered.groupby(by=['Course_Name', 'Session']):
        course_name, session = course[0]
        credit_hours = course[1].Credit_Hours.values.tolist()[0]
        for day_offered in course[1].Day.values.tolist():
            for period in periods_df[periods_df.Day_Name == day_offered].Period.values.tolist():
                for room in classrooms_df.Classroom:
                    aggregate_values.append((credit_hours / len(course[1])) * x_fcsrp[(professor.Faculty_Name, course_name, session, room, period)])
    model.add_constraint(model.sum(aggregate_values) <= professor.Workload_Credit_Hours)

#### Special Rules

1. Seniors courses start at 1:00 PM

In [11]:
### 800+ courses are taught after 1:00 PM
SENIOR_COURSE_NUMBER = 800
START_TIME = 13 # PM
for course in courses_offered[courses_offered.Course_Number >= SENIOR_COURSE_NUMBER].itertuples():
    for period in periods_df[(periods_df.Day_Name == course.Day) & (periods_df.Start_Time < START_TIME)].Period.values.tolist():
        for professor in preferences_df.Faculty_Name:
            for room in classrooms_df.Classroom:
                model.add_constraint(x_fcsrp[(professor, course.Course_Name, course.Session, room, period)] == 0)


2. No classes are taught on Friday after 4 PM

In [12]:
HOLIDAY = 'Friday'
END_TIME = 14.5
for course in courses_offered[courses_offered.Day == HOLIDAY].itertuples():
    for period in periods_df[(periods_df.Day_Name == HOLIDAY) & (periods_df.Start_Time >= END_TIME)].Period.values.tolist():
        for professor in preferences_df.Faculty_Name:
            for room in classrooms_df.Classroom:
                model.add_constraint(x_fcsrp[(professor, course.Course_Name, course.Session, room, period)] == 0)

#### Objective Function

In [13]:
total_preference = 0
for professor in preferences_df.itertuples():
    professor_df = pd.DataFrame(professor).T.drop(columns=0)
    professor_df.columns = preferences_df.columns
    for course in courses_offered.itertuples():
        course_code = '{}_{}'.format(course.Department_Code, str(course.Course_Number))
        preference_score = professor_df[course_code].values.tolist()[0]
        professor_name = professor_df.Faculty_Name.values.tolist()[0]
        for period in periods_df[periods_df.Day_Name == course.Day].Period.values.tolist():
            for room in classrooms_df.Classroom:
                total_preference += preference_score * x_fcsrp[(professor_name, course.Course_Name, course.Session, room, period)]

model.maximize(total_preference)
model.print_information()

Model: Course Scheduling
 - number of variables: 397440
   - binary=397440, integer=0, continuous=0
 - number of constraints: 915994
   - linear=915994
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


#### Solution

In [14]:
###############
# Solve Model #
###############
model.print_information()
solution = model.solve(log_output = True, time_limit = 120)
assert solution, "Solve Failed"
model.report()

Model: Course Scheduling
 - number of variables: 397440
   - binary=397440, integer=0, continuous=0
 - number of constraints: 915994
   - linear=915994
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               120
Tried aggregator 2 times.
MIP Presolve eliminated 843408 rows and 284080 columns.
MIP Presolve modified 1657 coefficients.
Aggregator did 71800 substitutions.
Reduced MIP has 786 rows, 41560 columns, and 301912 nonzeros.
Reduced MIP has 41560 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 8.14 sec. (5991.09 ticks)
Found incumbent of value 93.000000 after 11.31 sec. (7970.73 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 184 rows and 7940 columns.
Reduced MIP has 602 rows, 33620 columns, and 213998 nonzeros.
Reduced MIP has 33620 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve 

#### Post-processing

In [15]:
###################################
## View and Export model solution #
###################################
classes_assigned = []
professors_assigned = []
professors = []
courses = []
course_codes = []
rooms = []
sessions = []
periods = []
days = []
workloads = []
credits = []
room_capacity = []
course_enrollements = []
start_times = []
end_times = []
periods_covered = []

for course in courses_offered.groupby(by=['Course_Name', 'Session']):
    course_name, session = course[0]
    course_code = course[1].Course_Number.values.tolist()[0]
    course_credit = course[1].Credit_Hours.values.tolist()[0]
    course_enrollment = course[1].Max_Enrollement.values.tolist()[0]
    period_covered = course[1].Number_Periods_Covered.values.tolist()[0]
    for day_offered in course[1].Day.values.tolist():
        for professor in preferences_df.itertuples():
            for period in periods_df[periods_df.Day_Name == day_offered].Period.values.tolist():
                for room in classrooms_df.itertuples():
                    value = model.solution.get_value(x_fcsrp[(professor.Faculty_Name, course_name, session, room.Classroom, period)])
                    if value != 0:
                        professors.append(professor.Faculty_Name)
                        courses.append(course_name)
                        course_codes.append(course_code)
                        sessions.append(session)
                        rooms.append(room.Classroom)
                        days.append(day_offered)
                        periods.append(period)
                        workloads.append(professor.Workload_Credit_Hours)
                        credits.append(course_credit)
                        room_capacity.append(room.Capacity)
                        course_enrollements.append(course_enrollment)
                        start_times.append(periods_df[periods_df.Period == period].Start_Time.values.tolist()[0])
                        end_times.append(periods_df[periods_df.Period == period].End_Time.values.tolist()[0])
                        periods_covered.append(period_covered)
                        
                        if course_name not in classes_assigned:
                            classes_assigned.append(course_name)
                        if professor_name not in professors_assigned:
                            professors_assigned.append(professor_name)
    
print(f'Number of courses offered (regardless of number of sessions): {len(classes_assigned)}')
print(f'Number of courses offered (including different sessions): {len(courses_offered.groupby(by=["Course_Name", "Session"]))}')

all_assigned = (len(set(classes_assigned).difference(set(courses_offered.Course_Name.values.tolist()))) == 0) \
and (len(set(courses_offered.Course_Name.values.tolist()).difference(set(classes_assigned))) == 0)

print(f'All courses have been assigned = {all_assigned}')

class_schedule = pd.DataFrame({'Professor' : professors, 'Workload' : workloads, 'Course_Name': courses, 
                               'Course_Code' : course_codes, 'Credit' : credits, 'Session' : sessions, 
                               'Period' : periods, 'Period_Covered' : periods_covered, 'From' : start_times, 
                               'To': end_times, 'Day_Offered' : days, 'Max_Enrollement' : course_enrollements, 
                               'Classroom' : rooms, 'Room Capacity' : room_capacity})

class_schedule.sort_values(by=['Course_Name', 'Session', 'Day_Offered', 'Period'])

schedule_df = pd.DataFrame(columns=class_schedule.columns)
for course in class_schedule.itertuples():
    period_covered = course.Period_Covered
    start = course.From
    end = course.To
    period = course.Period
    next_frame = pd.DataFrame(course).T.drop(columns=0)
    next_frame.columns = class_schedule.columns
    schedule_df = schedule_df.append(next_frame, ignore_index=True)
    for index in range(period_covered - 1):
        start = end
        end += 0.5
        period += 1
        next_frame.From = start
        next_frame.To = end
        next_frame.Period = period
        schedule_df = schedule_df.append(next_frame, ignore_index=True)

### Write output
for course in schedule_df.groupby(by=['Professor']):
    display(course[1])
#schedule_df.to_excel(excel_writer='Classes Schedule_updated_2.xlsx', sheet_name='Report')

Number of courses offered (regardless of number of sessions): 34
Number of courses offered (including different sessions): 38
All courses have been assigned = True


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Period_Covered,From,To,Day_Offered,Max_Enrollement,Classroom,Room Capacity
108,Berlin Luxelly Londono Renteria,6,Insect Pest Management,810,3,1,18,3,16.5,17.0,Monday,20,1073,185
109,Berlin Luxelly Londono Renteria,6,Insect Pest Management,810,3,1,19,3,17.0,17.5,Monday,20,1073,185
110,Berlin Luxelly Londono Renteria,6,Insect Pest Management,810,3,1,20,3,17.5,18.0,Monday,20,1073,185
111,Berlin Luxelly Londono Renteria,6,Insect Pest Management,810,3,1,58,3,16.5,17.0,Wednesday,20,1073,185
112,Berlin Luxelly Londono Renteria,6,Insect Pest Management,810,3,1,59,3,17.0,17.5,Wednesday,20,1073,185
113,Berlin Luxelly Londono Renteria,6,Insect Pest Management,810,3,1,60,3,17.5,18.0,Wednesday,20,1073,185
160,Berlin Luxelly Londono Renteria,6,Introduction to Biological Control,621,3,1,31,3,13.0,13.5,Tuesday,40,1073,185
161,Berlin Luxelly Londono Renteria,6,Introduction to Biological Control,621,3,1,32,3,13.5,14.0,Tuesday,40,1073,185
162,Berlin Luxelly Londono Renteria,6,Introduction to Biological Control,621,3,1,33,3,14.0,14.5,Tuesday,40,1073,185
163,Berlin Luxelly Londono Renteria,6,Introduction to Biological Control,621,3,1,71,3,13.0,13.5,Thursday,40,1073,185


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Period_Covered,From,To,Day_Offered,Max_Enrollement,Classroom,Room Capacity
18,Brenda Oppert,3,Animal Health Entomology,305,3,2,29,3,12.0,12.5,Tuesday,50,1066,91
19,Brenda Oppert,3,Animal Health Entomology,305,3,2,30,3,12.5,13.0,Tuesday,50,1066,91
20,Brenda Oppert,3,Animal Health Entomology,305,3,2,31,3,13.0,13.5,Tuesday,50,1066,91
21,Brenda Oppert,3,Animal Health Entomology,305,3,2,69,3,12.0,12.5,Thursday,50,1066,91
22,Brenda Oppert,3,Animal Health Entomology,305,3,2,70,3,12.5,13.0,Thursday,50,1066,91
23,Brenda Oppert,3,Animal Health Entomology,305,3,2,71,3,13.0,13.5,Thursday,50,1066,91


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Period_Covered,From,To,Day_Offered,Max_Enrollement,Classroom,Room Capacity
30,Brian P. McCornack,9,Animal Health Entomology Laboratory,306,3,1,1,2,8.0,8.5,Monday,50,1066,91
31,Brian P. McCornack,9,Animal Health Entomology Laboratory,306,3,1,2,2,8.5,9.0,Monday,50,1066,91
32,Brian P. McCornack,9,Animal Health Entomology Laboratory,306,3,1,41,2,8.0,8.5,Wednesday,50,1066,91
33,Brian P. McCornack,9,Animal Health Entomology Laboratory,306,3,1,42,2,8.5,9.0,Wednesday,50,1066,91
34,Brian P. McCornack,9,Animal Health Entomology Laboratory,306,3,1,81,2,8.0,8.5,Friday,50,1066,91
35,Brian P. McCornack,9,Animal Health Entomology Laboratory,306,3,1,82,2,8.5,9.0,Friday,50,1066,91
42,Brian P. McCornack,9,Arthropod Vectors of Human Pathogens,849,3,1,12,2,13.5,14.0,Monday,20,1029,47
43,Brian P. McCornack,9,Arthropod Vectors of Human Pathogens,849,3,1,13,2,14.0,14.5,Monday,20,1029,47
44,Brian P. McCornack,9,Arthropod Vectors of Human Pathogens,849,3,1,52,2,13.5,14.0,Wednesday,20,1029,47
45,Brian P. McCornack,9,Arthropod Vectors of Human Pathogens,849,3,1,53,2,14.0,14.5,Wednesday,20,1029,47


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Period_Covered,From,To,Day_Offered,Max_Enrollement,Classroom,Room Capacity
6,Brian Spiesman,9,Advanced Integrative Behavioral Ecology,825,3,1,12,3,13.5,14.0,Monday,20,1066,91
7,Brian Spiesman,9,Advanced Integrative Behavioral Ecology,825,3,1,13,3,14.0,14.5,Monday,20,1066,91
8,Brian Spiesman,9,Advanced Integrative Behavioral Ecology,825,3,1,14,3,14.5,15.0,Monday,20,1066,91
9,Brian Spiesman,9,Advanced Integrative Behavioral Ecology,825,3,1,52,3,13.5,14.0,Wednesday,20,1066,91
10,Brian Spiesman,9,Advanced Integrative Behavioral Ecology,825,3,1,53,3,14.0,14.5,Wednesday,20,1066,91
11,Brian Spiesman,9,Advanced Integrative Behavioral Ecology,825,3,1,54,3,14.5,15.0,Wednesday,20,1066,91
48,Brian Spiesman,9,Conventional and Molecular Methods for Evaluat...,885,3,1,15,3,15.0,15.5,Monday,20,1029,47
49,Brian Spiesman,9,Conventional and Molecular Methods for Evaluat...,885,3,1,16,3,15.5,16.0,Monday,20,1029,47
50,Brian Spiesman,9,Conventional and Molecular Methods for Evaluat...,885,3,1,17,3,16.0,16.5,Monday,20,1029,47
51,Brian Spiesman,9,Conventional and Molecular Methods for Evaluat...,885,3,1,55,3,15.0,15.5,Wednesday,20,1029,47


...

,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Period_Covered,From,To,Day_Offered,Max_Enrollement,Classroom,Room Capacity
202,Raymond Cloyd,9,Plant Resistance to Insects,655,3,1,29,3,12.0,12.5,Tuesday,40,1029,47
203,Raymond Cloyd,9,Plant Resistance to Insects,655,3,1,30,3,12.5,13.0,Tuesday,40,1029,47
204,Raymond Cloyd,9,Plant Resistance to Insects,655,3,1,31,3,13.0,13.5,Tuesday,40,1029,47
205,Raymond Cloyd,9,Plant Resistance to Insects,655,3,1,69,3,12.0,12.5,Thursday,40,1029,47
206,Raymond Cloyd,9,Plant Resistance to Insects,655,3,1,70,3,12.5,13.0,Thursday,40,1029,47
207,Raymond Cloyd,9,Plant Resistance to Insects,655,3,1,71,3,13.0,13.5,Thursday,40,1029,47
232,Raymond Cloyd,9,Toxicology of Insecticides,857,4,1,11,3,13.0,13.5,Monday,20,1041,24
233,Raymond Cloyd,9,Toxicology of Insecticides,857,4,1,12,3,13.5,14.0,Monday,20,1041,24
234,Raymond Cloyd,9,Toxicology of Insecticides,857,4,1,13,3,14.0,14.5,Monday,20,1041,24
235,Raymond Cloyd,9,Toxicology of Insecticides,857,4,1,51,3,13.0,13.5,Wednesday,20,1041,24


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Period_Covered,From,To,Day_Offered,Max_Enrollement,Classroom,Room Capacity
150,Srinivas Kambhampati,6,Introduction to Arthropod Vectors of Human Pat...,649,5,1,9,2,12.0,12.5,Monday,40,1061,47
151,Srinivas Kambhampati,6,Introduction to Arthropod Vectors of Human Pat...,649,5,1,10,2,12.5,13.0,Monday,40,1061,47
152,Srinivas Kambhampati,6,Introduction to Arthropod Vectors of Human Pat...,649,5,1,29,2,12.0,12.5,Tuesday,40,1061,47
153,Srinivas Kambhampati,6,Introduction to Arthropod Vectors of Human Pat...,649,5,1,30,2,12.5,13.0,Tuesday,40,1061,47
154,Srinivas Kambhampati,6,Introduction to Arthropod Vectors of Human Pat...,649,5,1,49,2,12.0,12.5,Wednesday,40,1061,47
155,Srinivas Kambhampati,6,Introduction to Arthropod Vectors of Human Pat...,649,5,1,50,2,12.5,13.0,Wednesday,40,1061,47
156,Srinivas Kambhampati,6,Introduction to Arthropod Vectors of Human Pat...,649,5,1,69,2,12.0,12.5,Thursday,40,1061,47
157,Srinivas Kambhampati,6,Introduction to Arthropod Vectors of Human Pat...,649,5,1,70,2,12.5,13.0,Thursday,40,1061,47
158,Srinivas Kambhampati,6,Introduction to Arthropod Vectors of Human Pat...,649,5,1,89,2,12.0,12.5,Friday,40,1061,47
159,Srinivas Kambhampati,6,Introduction to Arthropod Vectors of Human Pat...,649,5,1,90,2,12.5,13.0,Friday,40,1061,47


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Period_Covered,From,To,Day_Offered,Max_Enrollement,Classroom,Room Capacity
96,Tania Kim,3,Insect Evolution,635,3,1,21,3,8.0,8.5,Tuesday,40,1066,91
97,Tania Kim,3,Insect Evolution,635,3,1,22,3,8.5,9.0,Tuesday,40,1066,91
98,Tania Kim,3,Insect Evolution,635,3,1,23,3,9.0,9.5,Tuesday,40,1066,91
99,Tania Kim,3,Insect Evolution,635,3,1,61,3,8.0,8.5,Thursday,40,1066,91
100,Tania Kim,3,Insect Evolution,635,3,1,62,3,8.5,9.0,Thursday,40,1066,91
101,Tania Kim,3,Insect Evolution,635,3,1,63,3,9.0,9.5,Thursday,40,1066,91


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Period_Covered,From,To,Day_Offered,Max_Enrollement,Classroom,Room Capacity
144,Weston Opitz,9,Insects of Stored Products,805,3,1,38,3,16.5,17.0,Tuesday,20,1069,22
145,Weston Opitz,9,Insects of Stored Products,805,3,1,39,3,17.0,17.5,Tuesday,20,1069,22
146,Weston Opitz,9,Insects of Stored Products,805,3,1,40,3,17.5,18.0,Tuesday,20,1069,22
147,Weston Opitz,9,Insects of Stored Products,805,3,1,78,3,16.5,17.0,Thursday,20,1069,22
148,Weston Opitz,9,Insects of Stored Products,805,3,1,79,3,17.0,17.5,Thursday,20,1069,22
149,Weston Opitz,9,Insects of Stored Products,805,3,1,80,3,17.5,18.0,Thursday,20,1069,22


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Period_Covered,From,To,Day_Offered,Max_Enrollement,Classroom,Room Capacity
90,William Morrison III,6,Insect Ecology,692,3,1,38,3,16.5,17.0,Tuesday,40,1066,91
91,William Morrison III,6,Insect Ecology,692,3,1,39,3,17.0,17.5,Tuesday,40,1066,91
92,William Morrison III,6,Insect Ecology,692,3,1,40,3,17.5,18.0,Tuesday,40,1066,91
93,William Morrison III,6,Insect Ecology,692,3,1,78,3,16.5,17.0,Thursday,40,1066,91
94,William Morrison III,6,Insect Ecology,692,3,1,79,3,17.0,17.5,Thursday,40,1066,91
95,William Morrison III,6,Insect Ecology,692,3,1,80,3,17.5,18.0,Thursday,40,1066,91
214,William Morrison III,6,Problems in Entomology,799,3,1,1,2,8.0,8.5,Monday,30,1052,60
215,William Morrison III,6,Problems in Entomology,799,3,1,2,2,8.5,9.0,Monday,30,1052,60
216,William Morrison III,6,Problems in Entomology,799,3,1,41,2,8.0,8.5,Wednesday,30,1052,60
217,William Morrison III,6,Problems in Entomology,799,3,1,42,2,8.5,9.0,Wednesday,30,1052,60


<center><a href='https://www.unomaha.edu/college-of-arts-and-sciences/mathematics/academics/ul-electives-spring.php' target='_blank'>Computational Operations Research</a>, Spring 2021, University of Nebraska at Omaha</center>